### Prepare Dataset

<p>
<a href='http://www.paulgraham.com/articles.html' target='_blank'>Paul Graham essays</a>
</p>

In [78]:
# import requests
# import bs4
# import json

# link = 'http://www.aaronsw.com/2002/feeds/pgessays.rss'
# res = requests.get(link)
# sp = bs4.BeautifulSoup(res.text)

# es_links = [[ele for ele in blck.get_text().split('\n') if ele][0]  for blck in sp.find_all('item') ]
# print('total essays:', len(es_links))

# def get_txt(link):
#     try:
#         res = requests.get(link)
#         sp = bs4.BeautifulSoup(res.text)
#         for br in sp('br'):
#             br.replace_with('\n')
#         return sp.find('body').find('table').find('font').get_text()
#     except:
#         print(f'failed: {link}')
# es_txts = [get_txt(link) for link in es_links]

# with open('./paulgraham_essays.json', 'w') as dest:
#     dest.write(json.dumps([ele for ele in es_txts if ele]))

## RNN 

In [79]:
import json
import numpy as np

### Load Dataset

In [194]:
with open('./paulgraham_essays.json', 'r') as f:
    data = json.loads(f.read())

In [195]:
txts = ""
for ele in data:
    txts += ele + '\n'

In [217]:
chars = list(set(txts))
char_sz,data_sz = len(chars), len(txts)
print(f'total chars: {data_sz}, unique chars: {char_sz}')

total chars: 1845065, unique chars: 103


In [218]:
ch_to_idx = {ch:idx for idx,ch in enumerate(chars)}
idx_to_ch = {idx:ch for ch,idx in ch_to_idx.items()}

In [219]:
h_sz = 100
sq_sz = 25
lr = 1e-1

In [220]:
wxh = np.random.randn(h_sz,char_sz)*0.01
whh = np.random.randn(h_sz,h_sz)*0.01
why = np.random.randn(char_sz,h_sz)*0.01
bh = np.zeros((h_sz, 1))
by = np.zeros((char_sz, 1))

In [221]:
wxh.shape,whh.shape,why.shape

((100, 103), (100, 100), (103, 100))

In [222]:
n,p=0,0
mwxh,mwhh,mwhy = np.zeros_like(wxh),np.zeros_like(whh),np.zeros_like(why)
mbh,mby = np.zeros_like(bh),np.zeros_like(by)
smooth_loss = -np.log(1.0/char_sz)*sq_sz # initial cross-entropy loss set based on uniform probs of chars

In [223]:
hprev = np.zeros((h_sz, 1))
inputs = [ch_to_idx[ch] for ch in txts[p:p+sq_sz]]
targets = [ch_to_idx[ch] for ch in txts[p+1:p+sq_sz+1]]

In [224]:
xs, hs, ys, ps = {}, {}, {}, {}
hs[-1] = np.copy(hprev)
loss = 0

In [225]:
for t in range(len(inputs)):
    xs[t] = np.zeros((char_sz, 1))
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(wxh, xs[t]) + np.dot(whh, hs[t-1])+ bh)
    ys[t] = np.dot(why, hs[t]) + by
    ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
    loss += -np.log(ps[t][targets[t], 0])e

In [226]:
loss

115.88311805955101

In [230]:
dwxh,dwhh,dwhy = np.zeros_like(wxh),np.zeros_like(whh),np.zeros_like(why)
dbh,dby = np.zeros_like(bh),np.zeros_like(by)
dhnext = np.zeros_like(hs[0])

In [ ]:
for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)

In [250]:
for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1
    dwhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(why.T, dy) + dhnext
    dhraw = (1-hs[t]*hs[t])*dh
    dbh += dhraw
    dwxh += np.dot(dhraw, xs[t].T)
    
    break